In [1]:
from ogb.lsc import MAG240MDataset
import numpy as np

In [2]:
dataset = MAG240MDataset(root = "/Volumes/Seagate Backup Plus Drive/Gradu/")

In [3]:
'''
edge_index is numpy.ndarray of shape (2, num_edges).
- first row: indices of source nodes (indexed by source node types)
- second row: indices of target nodes (indexed by target node types)
In other words, i-th edge connects from edge_index[0,i] to edge_index[1,i].
'''

edge_index_cites = dataset.edge_index('paper', 'paper')
edge_index_writes = dataset.edge_index('author', 'paper')

To create the test dataset to run on a regular computer, we begin by selecting an author and retrieving the papers they have contributed in. Next, we get the indexes of those papers, and retrieve both their citations and their authors. This cycle can be continued, and in the end we have an array of authors from which we can retrieve their affiliations. Then, we have an array of authors, an array of papers, three edgelists containing citations, writes and affiliations. We can then retrieve the required paper features from the array of papers and write the result into a file. 

We also need to select an article that actually has a label, since there are many unlabeled articles in the raw data.

In [4]:
print(np.where(dataset.paper_label==1))
print(edge_index_writes[:,edge_index_writes[1]==85]) # retrieve the author of paper 85 and use that as a starting point

(array([       85,      9052,      9166, ..., 121422531, 121494455,
       121634237]),)
[[1080565 1794309 7130696 7130697]
 [     85      85      85      85]]


In [11]:
paper_array = edge_index_writes[1,edge_index_writes[0]==1080565]
mask_cites = np.in1d(edge_index_cites[0],paper_array)
cites_array = edge_index_cites[:,mask_cites]

n = 1

for i in range(n):
    paper_array = np.concatenate((paper_array, np.unique(cites_array[1])))
    mask_cites = np.in1d(edge_index_cites[0],paper_array)
    cites_array = edge_index_cites[:,mask_cites]

mask_cites = np.in1d(edge_index_cites[0],paper_array)
cites_array = edge_index_cites[:,mask_cites]
paper_array = np.concatenate((paper_array, np.unique(cites_array[1])))
print(paper_array)


[       84        85    490533 ... 121707152 121712380 121748774]


In [12]:
paper_array_test = np.unique(paper_array)

In [17]:
edgelist_cites_test = cites_array
edgelist_cites_test

array([[       84,        84,        84, ..., 121156360, 121156360,
        121156360],
       [  2576935,  61873589,  90818614, ..., 102342995, 109155579,
        116684521]])

In [14]:
paper_label_test = dataset.paper_label[paper_array_test]

In [15]:
print(paper_array_test.shape)
print(edgelist_cites_test.shape)
print(paper_label_test.shape)


(11530,)
(2, 14021)
(11530,)


In [16]:
np.save('test_data_label_propagation/paper_array_test_label_propagation', paper_array_test)
np.save('test_data_label_propagation/edgelist_cites_test_label_propagation', edgelist_cites_test)
np.save('test_data_label_propagation/paper_label_test_label_propagation', paper_label_test)

Now, we have a smaller part of MAG-dataset that should be much easier to train with, before attempting anything with the dataset as a whole. 